In [1]:

list_destination=["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [2]:
import scrapy
import os
import datetime
import json
import pandas as pd

In [3]:
#checkign where we are, we should be in kayakproject
os.getcwd()

'c:\\Users\\nicol\\Desktop\\Kayakproject'

In [4]:
#going to main folder
os.chdir("main")
os.getcwd()

'c:\\Users\\nicol\\Desktop\\Kayakproject\\main'

In [5]:
#scraping the search results page from booking for the destinations. Spider used is in bookingmain.py, autothrottle is on and ROBOTSTXT_OBEY is False to avoid issues with booking.com
for destination in list_destination:
    os.environ["listdestination"]=destination
    !scrapy crawl bookingmain 

2022-05-03 12:26:32 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-05-03 12:26:32 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform Windows-10-10.0.19043-SP0
2022-05-03 12:26:32 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2022-05-03 12:26:32 [scrapy.extensions.telnet] INFO: Telnet Password: 53d733fa665f60ca
2022-05-03 12:26:32 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-05-03 12:26:32 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloader

In [6]:
#we scraped empty lines as the structure of each pages varies slighlty, so defining a function to clean the data and remove the nulls
def clean_nones(value):
    """
    Recursively remove all None values from dictionaries and lists, and returns
    the result as a new dictionary or list.
    """
    if isinstance(value, list):
        return [clean_nones(x) for x in value if x is not None]
    elif isinstance(value, dict):
        return {
            key: clean_nones(val)
            for key, val in value.items()
            if val is not None
        }
    else:
        return value

In [7]:
#we go to the results folder where the scrapped output files are
os.chdir("results")
os.getcwd()

In [9]:
#for everyfiles, opening then, cleaning them, and saving them
for files in os.listdir():
    with open(files) as json_file:
        data = json.load(json_file)
        datacleaned=clean_nones(data)
        datacleaned2=list(filter(None, datacleaned))
        with open(files, 'w') as f:
            json.dump(datacleaned2, f)

In [10]:
#now we go to the detailed folder, as scrapy does not allow 2 different spiders to be launched in the same folder
os.chdir("..")
os.chdir("..")
os.chdir("detailed")
os.getcwd()


'c:\\Users\\nicol\\Desktop\\Kayakproject\\main'

In [27]:
#we launched the detailed spiders, getting results for each hotel (gps and description). Spider is in bookingindividual.py
pathresults= os.path.dirname(os.getcwd())+"\\main\\results"
for file in os.listdir(pathresults):
    #we create an environ variable to path the list of files to the spider
    os.environ["files"]=str(pathresults+"\\"+file)
    !scrapy crawl bookingindividual

2022-05-02 16:30:45 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-05-02 16:30:45 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform Windows-10-10.0.19043-SP0
2022-05-02 16:30:45 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2022-05-02 16:30:45 [scrapy.extensions.telnet] INFO: Telnet Password: 6c5471312aaad27d
2022-05-02 16:30:45 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-05-02 16:30:46 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloader

In [35]:
#merge all files from the first spider


#create an empty dataframe
dataframemain= pd.DataFrame(columns = ["name","url","Rating"])


pathresults= os.path.dirname(os.getcwd())+"\\main\\results"
for file in os.listdir(pathresults):
    with open(os.path.dirname(os.getcwd())+"\\main\\results\\"+file) as json_file:
        data = json.load(json_file)
        dataframemain=pd.concat([dataframemain,pd.DataFrame(data)])

dataframemain.describe(include="all")



In [37]:
#Similar process for the detailed resulsts of the second spider, Creating an empty dataframe
dataframedetailed= pd.DataFrame(columns = ["url","description","bbox","Destination"])


pathresults= os.getcwd()+"\\resultsdetailed"
for file in os.listdir(pathresults):
    with open(pathresults+"\\"+file) as json_file:
        data = json.load(json_file)
        dataframedetailed=pd.concat([dataframedetailed,pd.DataFrame(data)])

dataframedetailed.head()

In [52]:
#a part of the url is missing from the second spider, so adding it
dataframedetailed["url"]=dataframedetailed["url"].apply(lambda x: x+"#hotelTmpl")

In [53]:
#Now that the url are similar, we can merge the 2 tables
dfbbox=pd.merge(dataframemain,dataframedetailed,on="url")

In [55]:
dfbbox.describe(include="all")

,name,url,Rating,description,bbox,Destination
count,883,883,861,883,883,883
unique,875,883,48,877,875,35
top,Hôtel Central,https://www.booking.com/hotel/fr/canal-aigues-...,"8,1",[Vous pouvez bénéficier d'une réduction Genius...,"7.33500722464803,48.0601503371431,7.3888857086...",Toulouse
freq,2,1,48,2,2,27


In [71]:
#convert bbox to gps. It's 2 set of lat and longitudes, so taking the average to have the center of the hotel instead of its overall area
dfbbox["Lat"]=dfbbox["bbox"].apply(lambda x: float(x.split(",")[1])/2+float(x.split(",")[3])/2)
dfbbox["Lon"]=dfbbox["bbox"].apply(lambda x: float(x.split(",")[0])/2+float(x.split(",")[2])/2)


In [73]:
dfbbox.head()

,name,url,Rating,description,bbox,Destination,Lat,Lon
0,Hotel Canal Aigues Mortes,https://www.booking.com/hotel/fr/canal-aigues-...,"8,6",[Vous pouvez bénéficier d'une réduction Genius...,"4.16998185644106,43.5550221035115,4.2196675347...",Aigues Mortes,43.572988,4.194825
1,La Villa Mazarin,https://www.booking.com/hotel/fr/la-villa-maza...,"9,1",[Vous pouvez bénéficier d'une réduction Genius...,"4.16691241748164,43.5470212030719,4.2165914962...",Aigues Mortes,43.564987,4.191752
2,Maison des Croisades,https://www.booking.com/hotel/fr/des-croisades...,"8,9",[Vous pouvez bénéficier d'une réduction Genius...,"4.1634570066567,43.5510319834269,4.21313939334...",Aigues Mortes,43.568997,4.188298
3,Hôtel Saint Louis,https://www.booking.com/hotel/fr/saint-louis-a...,"8,4",[Vous pouvez bénéficier d'une réduction Genius...,"4.16450314127171,43.549254254998,4.21418406165...",Aigues Mortes,43.567220,4.189344
4,Noemys Aigues-Mortes - ex Mona Lisa Royal Hôtel,https://www.booking.com/hotel/fr/le-royal-hote...,"6,9",[Featuring free WiFi and a seasonal outdoor sw...,"4.17297379558578,43.5584304713006,4.2226622861...",Aigues Mortes,43.576396,4.197818


In [76]:
#going back to kayak project and exporting the file
os.chdir("..")

dfbbox.to_csv("Hotels.csv")